In [29]:
import requests
import pandas as pd
import sqlite3

# importação das bibliotecas necessáriass ^

token = "46e3ffe3ef6f5a7ce21841ab6b11dd0f39317dae" 
# usado para acessar a API ^

In [30]:
def autentic(token):
     link = "https://api.brasil.io/v1"
     headers = {'Authorization': f'Token {token}'}
     rota = "dataset/cursos-prouni/cursos"

     # para uma rota dinâmica:
     tabela = "cursos"
     dataset = "cursos-prouni"

     rota = f"dataset/{dataset}/{tabela}/data"

     # verificando a conectividade

     response = requests.get(f"{link}/{rota}", headers=headers)

     if response.status_code == 200:
          print("Autenticação verificada com sucesso!😎")
          return response.json()['results']
     elif response.status_code == 401:
        print("Token inválido ou expirado!")
        return False
     elif response.status_code == 403:
        print("Sem permissão para acessar este recurso!")
        return False
     else:
        print(f"Erro inesperado: {response.status_code}")
        return False

In [31]:
# Conexão com o banco SQLite
conn = sqlite3.connect('cursos_prouni_br.db')
cursor = conn.cursor()

# Criando a tabela
cursor.execute('''
CREATE TABLE IF NOT EXISTS cursos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    uf_busca TEXT,
    cidade_busca TEXT,
    universidade TEXT,
    nome_curso TEXT,
    grau TEXT,
    mensalidade REAL,
    turno TEXT,
    bolsa_integral_ampla INTEGER,
    bolsa_integral_cotas INTEGER,
    bolsa_parcial_ampla INTEGER,
    bolsa_parcial_cotas INTEGER,
    nota_integral_ampla REAL,
    nota_integral_cotas REAL,
    nota_parcial_ampla REAL,
    nota_parcial_cotas REAL
)
''')
conn.commit()
conn.close()

In [32]:
# Alimentando o banco
conn = sqlite3.connect('cursos_prouni_br.db')
cursor = conn.cursor()

cursos = autentic(token)
for curso in cursos:
    uf_busca = curso['uf_busca']
    cidade_busca = curso['cidade_busca']
    universidade = curso['universidade_nome']  
    nome_curso = curso['nome']
    grau = curso['grau']
    mensalidade = curso['mensalidade']
    turno = curso['turno']  
    bolsa_integral_ampla = curso.get('bolsa_integral_ampla', None)
    bolsa_integral_cotas = curso.get('bolsa_integral_cotas', None)
    bolsa_parcial_ampla = curso.get('bolsa_parcial_ampla', None)
    bolsa_parcial_cotas = curso.get('bolsa_parcial_cotas', None)         
    nota_integral_ampla = curso.get('nota_integral_ampla', None)
    nota_integral_cotas = curso.get('nota_integral_cotas', None)
    nota_parcial_ampla = curso.get('nota_parcial_ampla', None)
    nota_parcial_cotas = curso.get('nota_parcial_cotas', None)

    cursor.execute('''
        INSERT INTO cursos (
            uf_busca, cidade_busca, universidade, nome_curso, grau, mensalidade, turno,
            bolsa_integral_ampla, bolsa_integral_cotas, bolsa_parcial_ampla, bolsa_parcial_cotas,
            nota_integral_ampla, nota_integral_cotas, nota_parcial_ampla, nota_parcial_cotas
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        uf_busca, cidade_busca, universidade, nome_curso, grau, mensalidade, turno,
        bolsa_integral_ampla, bolsa_integral_cotas, bolsa_parcial_ampla, bolsa_parcial_cotas,
        nota_integral_ampla, nota_integral_cotas, nota_parcial_ampla, nota_parcial_cotas
    ))


conn.commit()
conn.close()

print("Dados inseridos no banco de dados com sucesso!😎")

Autenticação verificada com sucesso!😎
Dados inseridos no banco de dados com sucesso!😎


## Alguns exemplos de consulta:

#### Consulta 1: Análise da Disponibilidade de Bolsas por Estado

###### Essa consulta visa analisar a distribuição de bolsas de estudo por estado, diferenciando bolsas integrais e parciais. O cálculo do percentual de bolsas integrais em relação ao número de cursos permite identificar quais estados priorizam mais a acessibilidade.


In [34]:
# Em SQL:
#SELECT
#     uf_busca AS estado,
#     COUNT(*) AS total_cursos,
#     SUM(bolsa_integral_ampla + bolsa_integral_cotas) AS total_bolsas_integral,
#     SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) AS total_bolsas_parcial,
#     (SUM(bolsa_integral_ampla + bolsa_integral_cotas) * 100.0 / 
#     NULLIF(COUNT(*), 0)) AS percentual_bolsas_integral
#FROM cursos
#GROUP BY uf_busca
#ORDER BY total_bolsas_integral DESC;

# Em python:

conn = sqlite3.connect("cursos_prouni_br.db")
cursor = conn.cursor()
cursor.execute(
     """SELECT 
     uf_busca AS estado,
     COUNT(*) AS total_cursos,
     SUM(bolsa_integral_ampla + bolsa_integral_cotas) AS total_bolsas_integral,
     SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) AS total_bolsas_parcial,
     (SUM(bolsa_integral_ampla + bolsa_integral_cotas) * 100.0 / 
     NULLIF(COUNT(*), 0)) AS percentual_bolsas_integral
     FROM cursos
     GROUP BY uf_busca
     ORDER BY total_bolsas_integral DESC;"""
)    
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

('MG', 882, 2304, 930, 261.2244897959184)
('BA', 582, 1209, 1221, 207.7319587628866)
('GO', 297, 753, 777, 253.53535353535352)
('CE', 222, 603, 138, 271.6216216216216)
('ES', 207, 453, 564, 218.84057971014494)
('MA', 147, 450, 1122, 306.1224489795918)
('MS', 213, 396, 159, 185.91549295774647)
('DF', 192, 393, 1467, 204.6875)
('AM', 72, 300, None, 416.6666666666667)
('AL', 57, 246, 54, 431.57894736842104)
('AP', 36, 129, None, 358.3333333333333)
('MT', 57, 84, 48, 147.3684210526316)
('AC', 36, 42, 39, 116.66666666666667)


#### Consulta 2: Identificação de Turnos Mais Populares

###### Identificar quais turnos (manhã, tarde, noite ou integral) são mais comuns entre os cursos oferecidos e como isso se relaciona com custos médios e a distribuição de bolsas.

In [35]:
# Em SQL:
#SELECT 
#turno, COUNT(*) AS total_cursos, AVG(mensalidade) AS mensalidade_media, SUM(bolsa_integral_ampla #+ bolsa_integral_cotas) AS total_bolsas_integral, SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) #AS total_bolsas_parcial FROM cursos GROUP BY turno ORDER BY total_cursos DESC;

# Em python:

conn = sqlite3.connect("cursos_prouni_br.db")
cursor = conn.cursor()
cursor.execute(
     """SELECT turno, COUNT(*) AS total_cursos, AVG(mensalidade) AS mensalidade_media, SUM(bolsa_integral_ampla + bolsa_integral_cotas) AS total_bolsas_integral, SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) AS total_bolsas_parcial FROM cursos GROUP BY turno ORDER BY total_cursos DESC;"""
)    
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

('Curso a Distância', 1539, 311.7809356725146, 3009, 606)
('Noturno', 1065, 751.0280281690141, 3462, 3912)
('Matutino', 342, 749.6043859649124, 822, 1923)
('Vespertino', 33, 706.5863636363636, 60, 48)
('Integral', 21, 871.5928571428572, 9, 30)


#### Análise das Universidades com Maior Quantidade de Bolsas Oferecidas

###### Identificar as universidades que mais contribuem para o programa de bolsas, tanto integrais quanto parciais, e analisar o custo médio dessas instituições.

In [36]:
# Em SQL:

#SELECT 
#    universidade,
#    COUNT(*) AS total_cursos,
#    SUM(bolsa_integral_ampla + bolsa_integral_cotas) AS total_bolsas_integral,
#    SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) AS total_bolsas_parcial,
#    AVG(mensalidade) AS mensalidade_media
#FROM cursos
#GROUP BY universidade
#ORDER BY total_bolsas_integral DESC
#LIMIT 10;

# Em python, da mesma forma:

conn = sqlite3.connect("cursos_prouni_br.db")
cursor = conn.cursor()
cursor.execute(
     """SELECT 
    universidade,
    COUNT(*) AS total_cursos,
    SUM(bolsa_integral_ampla + bolsa_integral_cotas) AS total_bolsas_integral,
    SUM(bolsa_parcial_ampla + bolsa_parcial_cotas) AS total_bolsas_parcial,
    AVG(mensalidade) AS mensalidade_media
FROM cursos
GROUP BY universidade
ORDER BY total_bolsas_integral DESC
LIMIT 10;"""
)    
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

('Universidade Pitágoras Unopar', 267, 963, None, 277.87640449438203)
('Universidade Anhanguera - UNIDERP', 150, 669, None, 257.6)
('Universidade Paulista - UNIP', 156, 405, None, 331.4301923076923)
('Faculdade de Ensino Superior da Cidade de Feira de Santana - FAESF/UNEF', 102, 243, 378, 295.0)
('Pontifícia Universidade Católica de Minas Gerais - PUC MINAS', 39, 219, None, 1003.3076923076923)
('Universidade de Uberaba - UNIUBE', 78, 201, None, 448.0573076923077)
('Universidade Salvador - UNIFACS', 36, 153, None, 506.6308333333333)
('Faculdade Pitágoras do Maranhão', 3, 123, None, 608.0)
('Centro Universitário Estácio de Ribeirão Preto - Estácio Ribeirão Pre - ESTÁCIO RIBEIRÃO PRE', 66, 108, None, 410.64681818181816)
('Centro Universitário de Caratinga - Unec - UNEC', 6, 72, None, 1000.0)


#### Análise da Relação entre Notas de Corte e Mensalidade

###### Analisar a relação entre as mensalidades médias, as notas de corte e as disparidades entre ampla concorrência e cotas em diferentes estados.

In [37]:
# Em SQL:

#SELECT 
#    uf_busca,
#    AVG(mensalidade) AS mensalidade_media,
#    AVG(nota_integral_ampla) AS nota_media_integral_ampla,
#    AVG(nota_integral_cotas) AS nota_media_integral_cotas,
#    AVG(nota_integral_ampla) - AVG(nota_integral_cotas) AS diferenca_media_ampla_cotas
#FROM cursos
#WHERE mensalidade IS NOT NULL 
#AND nota_integral_ampla IS NOT NULL 
#AND nota_integral_cotas IS NOT NULL
#GROUP BY uf_busca
#ORDER BY diferenca_media_ampla_cotas DESC;
#LIMIT 10;

# Em python, da mesma forma:

conn = sqlite3.connect("cursos_prouni_br.db")
cursor = conn.cursor()
cursor.execute(
     """SELECT 
    uf_busca,
    AVG(mensalidade) AS mensalidade_media,
    AVG(nota_integral_ampla) AS nota_media_integral_ampla,
    AVG(nota_integral_cotas) AS nota_media_integral_cotas,
    AVG(nota_integral_ampla) - AVG(nota_integral_cotas) AS diferenca_media_ampla_cotas
FROM cursos
WHERE mensalidade IS NOT NULL 
AND nota_integral_ampla IS NOT NULL 
AND nota_integral_cotas IS NOT NULL
GROUP BY uf_busca
ORDER BY diferenca_media_ampla_cotas DESC;"""
)    
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

('MS', 398.0291891891892, 567.4837837837838, 513.6972972972973, 53.786486486486524)
('GO', 462.0174545454546, 578.2654545454545, 542.9738181818183, 35.29163636363626)
('BA', 470.794262295082, 594.7226229508198, 567.1495081967213, 27.57311475409847)
('ES', 554.7326666666667, 610.888, 583.3213333333333, 27.56666666666672)
('MA', 537.3571428571429, 599.6028571428571, 572.6333333333333, 26.96952380952382)
('MT', 507.8211111111111, 557.1155555555555, 531.38, 25.735555555555493)
('AP', 624.0466666666666, 589.49, 567.1566666666666, 22.33333333333337)
('MG', 639.4462, 602.7550666666667, 580.7289333333333, 22.026133333333405)
('AM', 559.7069230769231, 579.3615384615384, 558.4476923076923, 20.91384615384618)
('AL', 477.5906666666666, 590.9613333333334, 575.8253333333333, 15.136000000000081)
('DF', 692.7446666666667, 589.8673333333334, 574.9406666666666, 14.926666666666733)
('CE', 557.3506818181818, 625.4431818181819, 612.8259090909091, 12.617272727272734)
('AC', 478.28749999999997, 590.285, 579.